In [24]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [25]:
text_samples = [
    "the quick brown fox",
    "hello world how are",
    "i love coding with",
    "this is a test",
    "machine learning is fun"
]

In [26]:
# Prepare the data
sequences = []
for sample in text_samples:
    words = sample.split()
    if len(words) == 4:
        sequences.append(words)

# Tokenize the words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sequences)
total_words = len(tokenizer.word_index) + 1


In [28]:
# Create input sequences and labels
input_sequences = []
labels = []
for seq in sequences:
    token_list = tokenizer.texts_to_sequences([seq[:3]])[0]
    input_sequences.append(token_list)
    label = tokenizer.texts_to_sequences([seq[3]])[0][0]
    labels.append(label)

In [29]:
# Pad sequences to make them uniform length (3 words)
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
labels = np.array(labels)


In [30]:
# Convert labels to categorical (one-hot encoding)
labels = tf.keras.utils.to_categorical(labels, num_classes=total_words)

In [31]:
# Build the RNN model
model = Sequential([
    Embedding(total_words, 10, input_length=max_sequence_len),
    SimpleRNN(32),
    Dense(total_words, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 3, 10)             200       
                                                                 
 simple_rnn_6 (SimpleRNN)    (None, 32)                1376      
                                                                 
 dense_6 (Dense)             (None, 20)                660       
                                                                 
Total params: 2236 (8.73 KB)
Trainable params: 2236 (8.73 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [32]:
# Train the model
model.fit(input_sequences, labels, epochs=100, verbose=1)

Epoch 1/100
1/1 [==============================] - 2s 2s/step - loss: 2.9957 - accuracy: 0.0000e+00
Epoch 2/100
1/1 [==============================] - 0s 8ms/step - loss: 2.9811 - accuracy: 0.0000e+00
Epoch 3/100
1/1 [==============================] - 0s 12ms/step - loss: 2.9666 - accuracy: 0.0000e+00
Epoch 4/100
1/1 [==============================] - 0s 9ms/step - loss: 2.9520 - accuracy: 0.0000e+00
Epoch 5/100
1/1 [==============================] - 0s 6ms/step - loss: 2.9374 - accuracy: 0.2000
Epoch 6/100
1/1 [==============================] - 0s 7ms/step - loss: 2.9227 - accuracy: 0.4000
Epoch 7/100
1/1 [==============================] - 0s 16ms/step - loss: 2.9078 - accuracy: 0.6000
Epoch 8/100
1/1 [==============================] - 0s 12ms/step - loss: 2.8927 - accuracy: 0.6000
Epoch 9/100
1/1 [==============================] - 0s 17ms/step - loss: 2.8774 - accuracy: 0.8000
Epoch 10/100
1/1 [==============================] - 0s 15ms/step - loss: 2.8618 - accuracy: 0.8000
Epoch 11/

In [33]:
# Function to predict the next word
def predict_next_word(text):
    token_list = tokenizer.texts_to_sequences([text.split()])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len, padding='pre')
    predicted = model.predict(token_list, verbose=0)
    predicted_index = np.argmax(predicted, axis=-1)[0]
    predicted_word = tokenizer.index_word[predicted_index]
    return predicted_word

In [34]:
# Test the model
test_sequences = [
    "the quick brown",
    "hello world how",
    "i love coding"
]

for seq in test_sequences:
    print(f"Input: {seq} -> Predicted next word: {predict_next_word(seq)}")

Input: the quick brown -> Predicted next word: fox
Input: hello world how -> Predicted next word: are
Input: i love coding -> Predicted next word: with
